# Optimization Models

## 1 Deterministic Linear Model: Upper Baseline

### 1.1 Mathematical Formulation

#### 1.1.1 Indices
\begin{align*}
i = 1,\dots,n &: \quad \text{set of stocks} \\
t = 1,\dots, T &: \quad \text{set of time periods} \\
\end{align*}

#### 1.1.2 Decision Variables
\begin{align*}
x_{it} &: \quad \text{amount of stock $i$ bought during period $t$} \\
z_{it} &: \quad \text{amount of stock $i$ owned at the end of period $t$} \\
c_{t} &: \quad \text{cash held at the end of period $t$} \\
\end{align*}

#### 1.1.3 Input Data and Parameters
\begin{align*}
P_{it} &: \quad \text{price of stock $i$ during period $t$} \\
C^0 &: \quad \text{initial cash on hand for investment in stocks} \\
\alpha &: \quad \text{commission paid per share traded} \\
\gamma &: \quad \text{maximum percentage of individual stock of total value of portfolio} \\
\delta &: \quad \text{limit on transaction amount} \\
\end{align*}

### 1.1.4 Model
\begin{align*}
\max_{c,x,t} & \quad \sum_{i=1}^n P_{i,T}z_{i,T}+c_T \\
\text{s.t.}
& \quad z_{i,1} = x_{i,1} & \quad \forall i \in [1,n] \\
& \quad (1+\alpha)\sum_{i=1}^n P_{i,1}x_{i,1} \leq C^0 \\
& \quad z_{i,t} = z_{i,t-1} + x_{i,t} & \quad \forall i \in [1,n], t \in [2,T] \\
& \quad c_{t} = c_{t-1} - \sum_{i=1}^n P_{it}x_{it} - \alpha \sum_{i=1}^n P_{it}\vert x_{it}\vert & \quad  t \in [2,T] \\
& \quad z_{i,t-1} + x_{i,t} \geq 0 & \quad \forall i \in [1,n], t \in [2,T] \\
& \quad -\delta\sum_{i=1}^n (z_{i,t-1}P_{i,t}) \leq x_{i,t}P_{i,t} \leq \delta\sum_{i=1}^n (z_{i,t-1}P_{i,t}) & \quad \forall i \in [1,n], t \in [2,T] \\
& \quad P_{i,t}z_{i,t} \leq \gamma\sum_{i=1}^n (P_{i,t}z_{i,t}) & \quad \forall i \in [1,n], t \in [2,T] \\
& \quad c_{t}, z_{i,t} \geq 0 & \quad \forall i \in [1,n], t \in [1,T] \\
\end{align*}

In [21]:
using JuMP, Gurobi

# Define the parameters
n = 5 # Number of stocks
T = 10 # Number of time periods
C0 = 100000 # Initial cash
alpha = 0.001 # Commission per share
gamma = 0.2 # Max percentage of individual stock
delta = 0.05 # Limit on transaction amount

# Randomly generated data for stock prices (for example purposes)
P = [91  16  19  20  59  30  15  83  20  60
 44  25  36  40  34  59  18  60  47  41
 51  56  74  11  46  26  35  69  55  25
 57  18  85  59  59  81  54   6  43  41
 32  14  89  87  90  49  79  15  60  73]

# Create a model with Gurobi Optimizer
model = Model(Gurobi.Optimizer)

# Decision Variables
@variable(model, x[1:n, 1:T]) # amount of stock i bought during period t
@variable(model, z[1:n, 1:T] >= 0) # amount of stock i owned at the end of period t
@variable(model, c[1:T] >= 0) # cash held at the end of period t
@variable(model, a[1:n, 1:T]) # auxilliary variable to linearize absolute value

# Objective: Maximize portfolio value at final period
@objective(model, Max, sum(P[i, T] * z[i, T] for i in 1:n) + c[T])

# Constraints
@constraint(model, [i=1:n], z[i, 1] == x[i, 1])
@constraint(model, (1 + alpha) * sum(P[i, 1] * x[i, 1] for i in 1:n) <= C0)
@constraint(model, [i=1:n, t in 2:T], z[i, t] == z[i, t-1] + x[i, t])
@constraint(model, c[1] == C0 - sum(P[i, 1] * x[i, 1] for i in 1:n) - alpha * sum(P[i, 1] * a[i, 1] for i in 1:n))
@constraint(model, [t in 2:T], c[t] == c[t-1] - sum(P[i, t] * x[i, t] for i in 1:n) - alpha * sum(P[i, t] * a[i, t] for i in 1:n))
@constraint(model, [i=1:n, t in 2:T], x[i,t] <= a[i,t])
@constraint(model, [i=1:n, t in 2:T], -x[i,t] <= a[i,t])
@constraint(model, [i=1:n, t in 2:T], z[i, t-1] + x[i, t] >= 0)
@constraint(model, [i=1:n, t in 2:T], -delta * sum(z[i, t-1] * P[i, t] for i in 1:n) <= x[i, t] * P[i, t])
@constraint(model, [i=1:n, t in 2:T],x[i, t] * P[i, t] <= delta * sum(z[i, t-1] * P[i, t] for i in 1:n))
@constraint(model, [i=1:n, t in 2:T], P[i, t] * z[i, t] <= gamma * sum(P[i, t] * z[i, t] for i in 1:n))

# Solve the model
optimize!(model)

# Output the results
println("Optimal Portfolio Value: ", objective_value(model))
println("Stocks to buy in each period: ")
println(value.(x))
println("Stocks owned at the end of each period: ")
println(value.(z))
println("Cash held at the end of each period: ")
println(value.(c))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 331 rows, 160 columns and 1304 nonzeros
Model fingerprint: 0x4a534346
Coefficient statistics:
  Matrix range     [6e-03, 9e+01]
  Objective range  [1e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+05, 1e+05]
Presolve removed 89 rows and 54 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible or unbounded model

User-callback calls 37, time in user-callback 0.00 sec


LoadError: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.